# Perfect captive

# Purpose
If the matematical model is not correct or too little data is available this may lead to paramter drift, so that the parameters in the matematical model changes depending on how the fitted data has been sampled.

This notebooks showcases the perfect case when you have the correct model and enought data in a captive test to identify the correct model.

# Methodology
* Sample data of forces from model
* Fit the parameters of the same model to this data
* Are the parameters correct?
* Is the simulation correct?

# Setup

In [ ]:
# %load imports.py
## Local packages:

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
np.set_printoptions(linewidth=150)

import numpy as np
import os
import matplotlib.pyplot as plt
#if os.name == 'nt':
#    plt.style.use('presentation.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl

from vessel_manoeuvring_models.symbols import *
from vessel_manoeuvring_models.parameters import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.models.regression import ForceRegression, results_summary_to_dataframe
from vessel_manoeuvring_models.models.diff_eq_to_matrix import DiffEqToMatrix
from vessel_manoeuvring_models.visualization.regression import show_pred, show_pred_captive
from vessel_manoeuvring_models.visualization.plot import track_plot,captive_plot

## Load models:
# (Uncomment these for faster loading):
import vessel_manoeuvring_models.models.vmm_abkowitz  as vmm 
#import vessel_manoeuvring_models.models.vmm_martin as vmm_simpler

from vessel_manoeuvring_models.models.vmm import ModelSimulator

from vessel_manoeuvring_models.data.wpcc import ship_parameters, df_parameters, ps, ship_parameters_prime, ps_ship, scale_factor
from vessel_manoeuvring_models.models.captive_variation import variate

In [ ]:
#format the book
import vessel_manoeuvring_models.visualization.book_format as book_format
book_format.set_style()

## Load model

In [ ]:
model = ModelSimulator.load('../models/model_VCT_abkowitz.pkl')

### Run a zigzag simulation with the model

In [ ]:
u0_=2
angle_deg = 35
result = model.zigzag(u0=u0_, angle=angle_deg)

In [ ]:
result.track_plot();
result.plot(compare=False);

In [ ]:
df_result = result.result.copy()
df_result_prime = model.prime_system.prime(df_result, U=df_result['U'])

## Parameter variation (captive test)

In [ ]:
len(model.parameters)

In [ ]:
variation_keys = ['u','v','r','delta']
df_inputs = variate(df=df_result_prime, variation_keys=variation_keys, N=3)
df_outputs = model.forces(df_inputs)
df_captive_all = pd.concat([df_inputs,df_outputs], axis=1)

In [ ]:
len(df_inputs)

In [ ]:
3**(len(variation_keys))

## Fit model

In [ ]:
reg_all = ForceRegression(vmm=model, data=df_captive_all)
display(reg_all.show_pred_X())
display(reg_all.show_pred_Y())
display(reg_all.show_pred_N())

### Create a simulation model from the regression model

In [ ]:
added_masses_ = {key:value for key,value in model.parameters.items() if 'dot' in key}
added_masses = pd.DataFrame(added_masses_, index=['prime']).transpose()

model_all = reg_all.create_model(added_masses=added_masses, ship_parameters=model.ship_parameters, 
                             ps=model.prime_system, control_keys=['delta'])

### Resimulate with the regressed model

In [ ]:
result_all = model_all.simulate(df_result)

In [ ]:
result_all.plot_compare();

In [ ]:
df_compare_parameters =pd.DataFrame()
df_compare_parameters['model'] = model.parameters
df_compare_parameters['model captive all'] = model_all.parameters


df_compare_parameters['model_abs'] = df_compare_parameters['model'].abs()
df_compare_parameters.sort_values(by='model_abs', ascending=False, inplace=True)
df_compare_parameters.drop(columns=['model_abs'], inplace=True)

df_compare_parameters = df_compare_parameters.divide(df_compare_parameters['model'], axis=0)

df_compare_parameters['dof'] = pd.Series(df_compare_parameters.index).apply(lambda x:x[0]).values

for dof, df_ in df_compare_parameters.groupby(by='dof', sort=False):
    fig,ax=plt.subplots()
    fig.set_size_inches(10,2)
    df_.plot(kind='bar', ax=ax)
    fig.suptitle(dof)